# Modeling SyriaTel’s Churn Rate
Hunter Owen, Oz Ahmed, Karim Oliver

# Project overview

The goals of this project was to provide the best model at predicting whether a customer will leave or not from the Syriatel Telecommunications company. 
Being such a competetive market and the recent access of communicating with people across the globe freely, Telecom companies are doing there best to retain customers. 
Along with a model that successfully predicts churn, we would like offer guidance as to what are the biggest features impacting customer churn. 
Our model should focus on predicting as many People who are leaving as possible which means trying to reduce false negatives(predicts them to stay, but they leave). This would give SyriaTel the chance to intervene and offer them deals that would get them to stay. 

# Data

The data we used can be found at Kaggle. 

The Target we are trying to predict is the churn and it has a high class embalance favoring people staying. 85% of the data set is people who statey and only 15% is people who left. Some of features included in this data are state, customer service calls, international plan, minutes, ect. 

The dataframe in this repo can be found here. 

# Goal

* Gather useful information from the dataset that can influence buissness decisions by SyriaTel
* Create a Model that can Predict churn with high recall score(focus on having least amount of false negatives). 

# Final Model & Results

#### Recall of 84% on hold out test 


#### Model: XGBoost Classifier
>Parameters:                                     
> * learning_rate = 0.01
> * max_depth = 4 
> * n_estimators = 500 
> * gamma = 0.3

>Pipeline:                                     
> * One Hot Encoder
> * Standard Scalar
> * Smote

<img src='figures/final_model.png' width=600/>

# Imports

In [11]:
#For dataframe and graphing
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


#Scikit learn libary and other similar one
from imblearn.pipeline import make_pipeline, Pipeline
from imblearn.over_sampling import SMOTE

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, f1_score, classification_report, plot_confusion_matrix, make_scorer, recall_score, accuracy_score, precision_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer, make_column_transformer
from xgboost import XGBClassifier 
import xgboost as xgb
import scipy

# For reading in custom functions
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Custom function
from src.mymods import model_output

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Exploring the Data and Early results

In [10]:
df = pd.read_csv('../../src/data/syriatel_customer_churn.csv')
df.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


<img src='figures/Internationalplan.png' width=600/>

<img src='figures/CustumorServiceCalls.png' width=600/>

<img src='figures/CostDayCharge.png' width=600/>

### Removing features

In [ ]:
y = pd.read_csv('../../src/data/y_dataframe.csv', index_col= 0, squeeze= True)
X = pd.read_csv('../../src/data/X_dataframe.csv', index_col= 0)

In [ ]:
X.drop(columns=['total day minutes', 'total eve minutes', 'total night minutes', 'total intl minutes', 'state'], axis=1, inplace= True)